#### Load LLM and device

In [1]:
from langchain_google_vertexai import ChatVertexAI
llm = ChatVertexAI(model="gemini-1.5-flash")

In [2]:
import torch

# Set the torch device
if torch.cuda.is_available():  # NVIDIA GPU
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # macOS
    device = torch.device("mps")
else:
    device = torch.device("cpu")  # CPU
print(f"Using {device.type} device")

Using cuda device


#### LongForm UQ

In [3]:
from uqlm import LongFormUQ

In [4]:
prompts = ["write a paragraph about Paul McCartney", "write a paragraph about John Lennon"]
sent_scorers = ["response_sent_entail", "response_sent_noncontradict", "response_sent_contrast_entail"]
claim_scorers = ["response_claim_entail", "response_claim_noncontradict", "response_claim_contrast_entail"]

In [5]:
# Sentence Level UQ
sent_lfuq = LongFormUQ(llm=llm, scorers=sent_scorers, device=device)
result = await sent_lfuq.generate_and_score(prompts=prompts, num_responses=2)
result.to_df()

Output()

,response,sampled_responses,prompt,sentence_set,response_sent_entail,response_sent_noncontradict,response_sent_contrast_entail
0,"Sir James Paul McCartney, a towering figure in...","[Sir Paul McCartney, a pivotal figure in music...",write a paragraph about Paul McCartney,"[Sir James Paul McCartney, a towering figure i...",0.313461,0.997370,0.976613
1,"John Lennon, a pivotal figure in the 20th cent...","[John Lennon, a pivotal figure in the 20th cen...",write a paragraph about John Lennon,"[John Lennon, a pivotal figure in the 20th cen...",0.710786,0.998759,0.996796


In [6]:
# Claim Level UQ
claim_lfuq = LongFormUQ(llm=llm, scorers=claim_scorers, device=device)
result = await claim_lfuq.generate_and_score(prompts=prompts, num_responses=2)
result.to_df()

Output()

,response,sampled_responses,prompt,claim_set,response_claim_entail,response_claim_noncontradict,response_claim_contrast_entail
0,"Sir James Paul McCartney, a pivotal figure in ...","[Sir James Paul McCartney, a pivotal figure in...",write a paragraph about Paul McCartney,[Sir James Paul McCartney is a pivotal figure....,0.774030,0.997366,0.959087
1,"John Lennon, a pivotal figure in the 20th cent...","[John Lennon, a pivotal figure in the 20th cen...",write a paragraph about John Lennon,"[John Lennon was a pivotal figure.\n, John Len...",0.717893,0.994498,0.950808


#### Decomposition

In [7]:
from uqlm.longform import ResponseDecomposer

In [8]:
rd = ResponseDecomposer(llm)

In [9]:
responses = [
    "Hello there! Today I visited Dr. Chauhan at his home. He was hanging his framed B.S., M.S., and Ph.D. degrees. It was fun!",
    "One time I went on a class trip to D.C. to visit the white house. While there, I felt in awe of the beutiful art and important people with titles such as Prof., Esq., and others."
]

In [10]:
rd.decompose_sentences(responses=responses)

[['Hello there!',
  'Today I visited Dr. Chauhan at his home.',
  'He was hanging his framed B.S., M.S., and Ph.D. degrees.',
  'It was fun!'],
 ['One time I went on a class trip to D.C. to visit the white house.',
  'While there, I felt in awe of the beutiful art and important people with titles such as Prof., Esq., and others.']]

In [11]:
await rd.decompose_claims(responses=responses)

[['I visited Dr. Chauhan.\n',
  'I visited Dr. Chauhan at his home.\n',
  'Dr. Chauhan was hanging his degrees.\n',
  'Dr. Chauhan was hanging his B.S. degree.\n',
  'Dr. Chauhan was hanging his M.S. degree.\n',
  'Dr. Chauhan was hanging his Ph.D. degree.\n',
  "Dr. Chauhan's degrees were framed.\n",
  'It was fun.\n'],
 ['I went on a class trip.\n',
  'The class trip was to D.C.\n',
  'I visited the white house.\n',
  'I felt in awe.\n',
  'The art was beautiful.\n',
  'The people were important.\n',
  'The people had titles.\n',
  'The titles included Prof..\n',
  'The titles included Esq..\n',
  'The titles included others.\n\n']]